In [1]:
import pandas as pd
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import AdamW, WarmUp, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
df = pd.read_csv("dataset/Preprocessed_Train_Dataset.csv")
df = df[:45000]

In [3]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

In [4]:
special_tokens_dict = {'pad_token': '<pad>', 'bos_token': '<bos>', 'eos_token': '<eos>', 
                       'additional_special_tokens': ['<PAGESTART>', '<PAGEEND>', '<SECTIONSTART>', '<SECTIONEND>',
                                                     '<TABLESTART>','<TABLEEND>','<CELLSTART>','<CELLEND>','<COLHEADERSTART>',
                                                     '<COLHEADEREND>','<ROWHEADERSTART>','<ROWHEADEREND>']}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print('We have added', num_added_toks, 'tokens')
model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))

We have added 14 tokens


Embedding(32114, 768)

In [5]:
MAXLENI = 400
MAXLENO = 200

class tottodataset(Dataset):
    def __init__(self,df,tokenizer):
        self.sentence=df['sentence']
        self.table=df['table']
        self.tokenizer=tokenizer

    def __len__(self):
        return len(self.sentence)
  
    def __getitem__(self,idx):
        inp=(self.table[idx]+'</s>').replace("<page_title>", "<PAGESTART>").replace("</page_title>", "<PAGEEND>") \
                                    .replace("<section_title>", "<SECTIONSTART>").replace("</section_title>", "<SECTIONEND>") \
                                    .replace("<table>", "<TABLESTART>").replace("</table>", "<TABLEEND>") \
                                    .replace("<cell>", "<CELLSTART>").replace("</cell>", "<CELLEND>") \
                                    .replace("<col_header>", "<COLHEADERSTART>").replace("</col_header>", "<COLHEADEREND>") \
                                    .replace("<row_header>", "<ROWHEADERSTART>").replace("</row_header>", "<ROWHEADEREND>")
        out=self.sentence[idx]+'</s>'
        inp_tokens=self.tokenizer.encode_plus(inp, padding="max_length", max_length=MAXLENI, truncation=True)
        out_tokens=self.tokenizer.encode_plus(out, padding="max_length", max_length=MAXLENO, truncation=True)
        inp_id=inp_tokens.input_ids
        out_id=out_tokens.input_ids
        inp_mask=inp_tokens.attention_mask
        out_mask=out_tokens.attention_mask
        labels=out_tokens.input_ids.copy()
        labels=[-100  if x==self.tokenizer.pad_token_id else x for x in labels]

        return {
        "table_text":inp,
        "sentence":out,
        "input_ids":torch.tensor(inp_id, dtype=torch.long),
        "input_attention_mask":torch.tensor(inp_mask, dtype=torch.long),
        "decoder_input_ids":torch.tensor(out_id, dtype=torch.long),
        "decoder_attention_mask":torch.tensor(out_mask, dtype=torch.long),
        "labels":torch.tensor(labels, dtype=torch.long)
    }

In [6]:
train_df = df[:41000]
val_df = df[41000:].reset_index()

In [7]:
train_dataset = tottodataset(train_df,tokenizer)
val_dataset = tottodataset(val_df,tokenizer)

In [8]:
train_dataloader=DataLoader(train_dataset, batch_size=4, num_workers=2, shuffle=False)
val_dataloader=DataLoader(val_dataset, batch_size=4, num_workers=2, shuffle=False)

In [9]:
dataloaders = {'train':train_dataloader, 'eval':val_dataloader}
dataset_sizes = {'train':len(train_dataset), 'eval':len(val_dataset)}

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [11]:
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

/home/191it220/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
!nvidia-smi

Mon Apr 25 14:50:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla M40 24GB      Off  | 00000000:04:00.0 Off |                    0 |
| N/A   58C    P0    62W / 250W |   2597MiB / 22945MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40 24GB      Off  | 00000000:82:00.0 Off |                    0 |
| N/A   

In [13]:
def train_fn(model,optimizer,scheduler,num_epochs):
    since=time.time()
    best_wts=copy.deepcopy(model.state_dict())
    best_loss=float('inf')
    for epoch in range(num_epochs):
        print(f'Epoch:{epoch}/{num_epochs}')
        print('-'*10)
        
        for mode in ['train','eval']:
            if mode=='train':
                model.train()
            elif mode=='eval':
                model.eval()
            
            running_loss=0.0
            
            for data in tqdm(dataloaders[mode]):
                input_ids = data["input_ids"].to(device, dtype=torch.long)
                labels = data['labels'].to(device, dtype=torch.long)
            
                optimizer.zero_grad()
                with torch.set_grad_enabled(mode=='train'):
                    outputs=model(
                                input_ids =input_ids,
                                labels = labels
                            )
                    loss, logits=outputs[:2]
                    
                    if mode=='train':
                        loss.backward()
                        optimizer.step()                    
                    running_loss += loss.item()

            if mode == 'train':
                scheduler.step()
                
            epoch_loss=running_loss/dataset_sizes[mode]
            
            print('{} Loss: {:.4f} '.format(
                mode, epoch_loss))
            
            if mode=='eval' and epoch_loss<best_loss:
                best_wts=copy.deepcopy(model.state_dict())
                best_loss=epoch_loss
            
            print()

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best val loss: {:4f}'.format(best_loss))
        model.load_state_dict(best_wts)
        torch.save(model, "dataset/T5model_epoch_7")
    
    return model

In [14]:
history = train_fn(model,optimizer,scheduler,num_epochs=7)

Epoch:0/7
----------


  0%|          | 0/10250 [00:00<?, ?it/s]

train Loss: 0.2892 



  0%|          | 0/1000 [00:00<?, ?it/s]

eval Loss: 0.2411 

Training complete in 132m 22s
Best val loss: 0.241068
Epoch:1/7
----------


...

eval Loss: 0.2273 

Training complete in 789m 6s
Best val loss: 0.227302
Epoch:6/7
----------


  0%|          | 0/10250 [00:00<?, ?it/s]

train Loss: 0.1808 



  0%|          | 0/1000 [00:00<?, ?it/s]

eval Loss: 0.2271 

Training complete in 920m 24s
Best val loss: 0.227132


In [15]:
torch.save(model, "T5model_epoch_7")